<a href="https://colab.research.google.com/github/thenewlegend/AITfES/blob/main/AITfES(m).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-genai
import os
import google.genai as genai
from google.genai import types
import json
from google.colab import userdata

In [14]:
query_text = "What is the capital of rome  in energy engineering terms?" # @param {"type":"string","placeholder":"-------"}

def analyze_energy_query(query: str):
    return f"RESULT: The analysis confirms this is a valid energy engineering query: {query}. Proceeding with detailed response."

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
model_name = 'gemini-2.5-flash'
model = genai.Client()

SYSTEM_INSTRUCTION = "You are a highly specialized AI assistant. Your sole purpose is to determine if the user's question relates to troubeshooting energy engineering related equipments or processes. If related, you MUST call the 'analyze_energy_query' tool. If NOT related, you MUST respond with the exact text: 'Error: Query out of scope. I only handle Energy Engineering troubleshooting.'"


tools = [analyze_energy_query]

query = query_text
print(f"\n--- Testing Full Query: '{query}' ---")

response = model.models.generate_content(
    model=model_name,
    contents=[query],
    config=types.GenerateContentConfig(
        tools=tools,
        system_instruction=SYSTEM_INSTRUCTION
    )
  )
if response.function_calls:
    function_call = response.function_calls[0]

    # 1. Execute the tool function
    tool_output = globals()[function_call.name](**dict(function_call.args))

    # 2. Call the model again with the tool output
    final_response = model.models.generate_content(
        model=model_name,
        contents=[
            query,
            types.Part.from_function_response(
                name=function_call.name,
                response={'result': tool_output}
            )
        ],
        config=types.GenerateContentConfig(
            tools=tools,
            system_instruction=SYSTEM_INSTRUCTION
        )
    )
    print("ACTION: Function Called and Executed.")
    print(f"FINAL MODEL RESPONSE: {final_response.text}")
else:
    print("ACTION: Query Rejected.")
    print(f"FINAL MODEL RESPONSE: {response.text}")


--- Testing Full Query: 'What is the capital of rome  in energy engineering terms?' ---


ACTION: Query Rejected.
FINAL MODEL RESPONSE: Error: Query out of scope. I only handle Energy Engineering troubleshooting.
